In [6]:
from youtube_transcript_api import YouTubeTranscriptApi
import textwrap

# Step 1: Get the transcript
def get_transcript(video_id):
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
    # Try to get a manually created transcript
    try:
        transcript = transcript_list.find_manually_created_transcript(transcript_list._manually_created_transcripts.keys())
    except:
        # If no manually created transcript is found, get an automatically generated one
        transcript = transcript_list.find_generated_transcript(transcript_list._generated_transcripts.keys())
    transcript_data = transcript.fetch()
    transcript_text = " ".join([item['text'] for item in transcript_data])
    return transcript_text

# Step 2: Divide the transcript into sections
def divide_transcript(transcript):
    sections = textwrap.wrap(transcript, 15000)
    return sections

# Step 3: Create the GPT prompts
def create_prompts(sections):
    prompts = []
    total_sections = len(sections)
    for i, section in enumerate(sections):
        start_marker = f"[START CHUNK {i+1}/{total_sections}]"
        end_marker = f"[END CHUNK {i+1}/{total_sections}], now resume this chunk with a bullet list"
        if i == 0:  # This is the first chunk
            prompts.append(f"I will send you a transcription of someone speaking. Help me understand what he said; resume with a bullet list.\n{start_marker}\n{section}\n{end_marker}")
        else:
            prompts.append(f"{start_marker}\n{section}\n{end_marker}")
    return prompts

# Use the functions
video_id = "D81Xh9FSjG8"
transcript = get_transcript(video_id)
sections = divide_transcript(transcript)
prompts = create_prompts(sections)
